# 🎧 Spotify MP3 Converter

Converte músicas, álbuns e playlists do **Spotify** para arquivos **MP3** automaticamente, utilizando o **YouTube** como fonte — tudo direto no **Google Colab**.

---

👤 **Autor:** Kira_Ghoul  
📅 **Projeto:** Spotify MP3 Converter  
💻 **Plataforma:** Google Colab  

---

🔗 **Acesse o Projeto no Colab:**  
<a href="https://colab.research.google.com/drive/18E5p9n6z6HHDA-Q86EKtw1gXNEedz-lW?usp=sharing" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

---

⚠️ **Aviso Legal:** Este projeto é apenas para fins educacionais.  
Respeite os direitos autorais das plataformas envolvidas.

In [ ]:
# ==========================================
# ⚡ Spotify → YouTube MP3 Downloader (Colab)
# ==========================================

!pip install -q yt-dlp spotipy mutagen rapidfuzz python-dotenv

# ---------- Imports ----------
import os, shutil, glob, requests, queue, threading
import yt_dlp, spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from rapidfuzz import fuzz
from mutagen.mp3 import MP3
from mutagen.id3 import ID3, TIT2, TPE1, TALB, APIC
from google.colab import files
from time import sleep
from dotenv import load_dotenv

# ---------- Upload do ficheiro .env com chaves ----------
env_path = None
print("📁 Faça upload do arquivo keys.env com suas chaves do Spotify:")
try:
    uploaded = files.upload()
    if uploaded:
        env_path = list(uploaded.keys())[0]
        print(f"✅ keys.env carregado: {env_path}")
except:
    print("⚠️ Nenhum arquivo enviado. Não será possível autenticar o Spotify sem chaves.")

# Carrega as variáveis do .env
if env_path:
    load_dotenv(env_path)

SPOTIFY_CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
SPOTIFY_CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET")

if not SPOTIFY_CLIENT_ID or not SPOTIFY_CLIENT_SECRET:
    raise ValueError("❌ SPOTIFY_CLIENT_ID ou SPOTIFY_CLIENT_SECRET não foram fornecidos!")

# ---------- Inicializa Spotify ----------
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=SPOTIFY_CLIENT_ID,
    client_secret=SPOTIFY_CLIENT_SECRET
))

# ---------- Cookies opcional ----------
cookies_path = None
print("📁 Opcional: faça upload do arquivo cookies do YouTube (cookies.txt).")
print("   Ou clique em 'Cancelar' para continuar sem cookies.")
try:
    uploaded = files.upload()
    if uploaded:
        cookies_path = list(uploaded.keys())[0]
        print(f"✅ Cookies carregados: {cookies_path}")
except:
    print("⚠️ Nenhum cookies enviado. Continuando sem cookies...")

# ---------- Variável global para contador ----------
counter_lock = threading.Lock()
global_index = 1  # contador global das músicas

# ---------- Funções auxiliares ----------
def tag_mp3(path, title, artist, album, img_url=None):
    """Adiciona tags e capa à MP3"""
    try:
        audio = MP3(path, ID3=ID3)
        audio.add_tags()
    except:
        pass
    audio.tags.add(TIT2(encoding=3, text=title))
    audio.tags.add(TPE1(encoding=3, text=artist))
    audio.tags.add(TALB(encoding=3, text=album))
    if img_url:
        try:
            img = requests.get(img_url, timeout=5).content
            audio.tags.add(APIC(encoding=3, mime='image/jpeg', type=3, desc='Cover', data=img))
        except:
            pass
    audio.save()

def simple_match(track_name, yt_title):
    return fuzz.partial_ratio(track_name.lower(), yt_title.lower()) > 70

def download_worker(q, folder):
    global global_index

    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': f'{folder}/%(title)s.%(ext)s',
        'quiet': True,
        'no_warnings': True,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192'
        }]
    }

    if cookies_path:
        ydl_opts['cookies'] = cookies_path

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        while True:
            try:
                track = q.get(timeout=2)
            except queue.Empty:
                break

            title = track['name']
            artist = track['artists'][0]['name']
            album = track['album']['name']
            cover = track['album']['images'][0]['url'] if track['album']['images'] else None
            q_str = f"{title} {artist}"

            with counter_lock:
                index = global_index
                global_index += 1

            print(f"🎵 Baixando {index}º: {title} - {artist}")

            try:
                results = ydl.extract_info(f"ytsearch3:{q_str}", download=False)
                entry = next((e for e in results['entries'] if simple_match(title, e['title'])), results['entries'][0])
                ydl.download([entry['webpage_url']])
                mp3 = max(glob.glob(f"{folder}/*.mp3"), key=os.path.getmtime)
                tag_mp3(mp3, title, artist, album, cover)
            except yt_dlp.utils.DownloadError as e:
                if "Sign in to confirm you're not a bot" in str(e) and not cookies_path:
                    print("⚠️ YouTube bloqueou. Faça upload do cookies.txt para continuar.")
                else:
                    print(f"⚠️ {title}: {e}")
            finally:
                q.task_done()

def download_spotify(url, max_threads=5):
    if "playlist" in url:
        info = sp.playlist(url)
        name = info['name']
        results = sp.playlist_items(url)
        tracks = []
        while results:
            tracks.extend([i['track'] for i in results['items'] if i['track']])
            results = sp.next(results)
    elif "album" in url:
        info = sp.album(url)
        name = info['name']
        tracks = info['tracks']['items']
        for t in tracks:
            t['album'] = info
    elif "track" in url:
        info = sp.track(url)
        name = info['name']
        tracks = [info]
    else:
        print("❌ Link Spotify inválido.")
        return

    print(f"\n🎧 Encontradas {len(tracks)} músicas em '{name}'\n")

    folder = f"/content/{name.replace('/', '_')}"
    os.makedirs(folder, exist_ok=True)

    q = queue.Queue()
    for t in tracks:
        q.put(t)

    num_threads = min(max_threads, len(tracks))
    print(f"🚀 Iniciando {num_threads} threads de download...\n")

    threads = []
    for _ in range(num_threads):
        t = threading.Thread(target=download_worker, args=(q, folder))
        t.start()
        threads.append(t)
        sleep(0.5)

    for t in threads:
        t.join()

    zip_path = f"/content/{name.replace('/', '_')}.zip"
    shutil.make_archive(zip_path[:-4], 'zip', folder)
    print(f"\n✅ Downloads completos! ZIP: {zip_path}")

    files.download(zip_path)
    shutil.rmtree(folder, ignore_errors=True)

spotify_link = input("🎵 Cole o link da playlist/álbum/música do Spotify: ").strip()
download_spotify(spotify_link, max_threads=5)